In [127]:
# Load Data

import torch
import numpy as np

#Train data refinement
train_data = np.loadtxt("Train_Data.txt", delimiter=',', dtype=str)

# delete data w. missing val
missing_val = (train_data == '?') 
missing_row = np.any(missing_val, axis=1)
 
final_data = train_data[ ~ missing_row]


# delete ? / change class code
final_data = np.delete(final_data, 0, 1)
final_data = final_data.astype(np.float32)

X = final_data[:, :-1]
Y = final_data[:, -1]

Y[Y==4] = 1
Y[Y==2] = 0

X_mean = X.mean(axis=0)
X_std = X.std(axis=0)

X = (X - X_mean) / X_std

X_tensor = torch.from_numpy(X).float()
Y_tensor = torch.from_numpy(Y).float().view(-1, 1)



#Test data refinement
test_data = np.loadtxt("Test_Data.txt", delimiter=',', dtype=str)

# delete data w. missing val
missing_testval = (test_data == '?') 
missing_testrow = np.any(missing_testval, axis=1)

final_testdata = test_data[ ~ missing_testrow]


# delete ? / change class code
final_testdata = np.delete(final_testdata, 0, 1)
final_testdata = final_testdata.astype(np.float32)

Xt = final_testdata[:, :-1]
Yt = final_testdata[:, -1]

Yt[Yt==4] = 1
Yt[Yt==2] = 0

Xt = (Xt - X_mean) / X_std

Xt_tensor = torch.from_numpy(Xt).float()
Yt_tensor = torch.from_numpy(Yt).float().view(-1, 1)



from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim


dataset = TensorDataset(X_tensor, Y_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define Model
class LogisticRegression(nn.Module):
    def __init__(self) :
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(9, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

r = 0.02 # regularization strength
model = LogisticRegression()
criterion = nn.BCELoss()
Optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay = r)


# Train Model
num_epochs = 400
loss_history = []
result = [ ]

for epoch in range(num_epochs):
    for X_batch, Y_batch in train_loader:
        Optimizer.zero_grad()
        Y_val = model(X_batch)
        loss = criterion(Y_val, Y_batch)
        loss.backward()
        Optimizer.step()

    
    current_loss = loss.item()
    loss_history.append(current_loss)

#Model Evaluation
model.eval()
with torch.no_grad() :
    #train data
    train_output = model(X_tensor)
    train_bool = (train_output >=0.5).float()
    train_accuracy = (train_bool == Y_tensor).sum().item() / len(Y_tensor) * 100 #백분율
    
    #test data
    test_output = model(Xt_tensor)
    test_bool = (test_output >=0.5).float()
    test_accuracy = (test_bool == Yt_tensor).sum().item() / len(Yt_tensor) * 100 #백분율

result_new = {" accuracy": train_accuracy, "test accuracy" : test_accuracy , "lambda" : r, "epochs" : num_epochs}
result.append(result_new)
print(f'accuracy : {train_accuracy:.3f}%, rambda : {r}', f'test accuracy : {test_accuracy:.3f}%')



accuracy : 96.747%, rambda : 0.02 test accuracy : 98.990%


In [128]:
print(result)

[{' accuracy': 96.74657534246576, 'test accuracy': 98.98989898989899, 'lambda': 0.02, 'epochs': 400}]
